# Instancing Spark

Import the PySpark library and the `SparkSession` class

In [1]:
import pyspark
from pyspark.sql import SparkSession

Instance a session

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/02/27 09:09:49 WARN Utils: Your hostname, Desktop-Gar resolves to a loopback address: 127.0.1.1; using 172.25.243.204 instead (on interface eth0)
24/02/27 09:09:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 09:09:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/27 09:09:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Read data

Download some large data

In [3]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

Read the data. Note that the datatypes are not inferred; everything is a string.

In [3]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

In [4]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [5]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

Read the schema

In [6]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [7]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)



# Creating a schema with Pandas

Write the first 1001 lines of the CSV file to another file

In [8]:
!zcat < fhvhv_tripdata_2021-01.csv.gz | head -n 1001  > head.csv


gzip: stdout: Broken pipe


Count the lines in the new CSV file

In [9]:
!wc -l head.csv

1001 head.csv


We will now use Pandas to infer the datatypes. Let's import Pandas and read the small CSV file.

In [10]:
import pandas as pd

In [11]:
df_pandas = pd.read_csv('head.csv')

Let's see the inferred types.

In [12]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

Spark can create dataframes from Pandas dataframes. Let's see how Spark converts it and check the datatypes as defined in Spark.

In [13]:
spark.createDataFrame(df_pandas).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [14]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

This datatype description is from Scala, which is the language that Spark is created with. Let's clean up the output and create our usable schema from it.

In [15]:
# Don't run this block!
'''
StructType(
    List(
        StructField(hvfhs_license_num,StringType,true),
        StructField(dispatching_base_num,StringType,true),
        StructField(pickup_datetime,StringType,true),
        StructField(dropoff_datetime,StringType,true),
        StructField(PULocationID,LongType,true),
        StructField(DOLocationID,LongType,true),
        StructField(SR_Flag,DoubleType,true)
    )
)
'''

'\nStructType(\n    List(\n        StructField(hvfhs_license_num,StringType,true),\n        StructField(dispatching_base_num,StringType,true),\n        StructField(pickup_datetime,StringType,true),\n        StructField(dropoff_datetime,StringType,true),\n        StructField(PULocationID,LongType,true),\n        StructField(DOLocationID,LongType,true),\n        StructField(SR_Flag,DoubleType,true)\n    )\n)\n'

The timestamps were not originally inferred by Pandas but we know which fields should be datatypes. We will convert longs (`int64`) to integers for memory optimization, and convert `SR_Flag` to string.

In [15]:
from pyspark.sql import types

In [16]:
schema = types.StructType(
    [
        types.StructField('hvfhs_license_num', types.StringType(), True),
        types.StructField('dispatching_base_num', types.StringType(), True),
        types.StructField('pickup_datetime', types.TimestampType(), True),
        types.StructField('dropoff_datetime', types.TimestampType(), True),
        types.StructField('PULocationID', types.IntegerType(), True),
        types.StructField('DOLocationID', types.IntegerType(), True),
        types.StructField('SR_Flag', types.StringType(), True)
    ]
)

With this new schema, we can now create a dataframe with inferred datatypes.

In [17]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

In [18]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [19]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

# Partitions

We will now create 24 partitions in our dataframe

In [20]:
df = df.repartition(24)

Let's parquetize the dataframe. This will create 24 smaller parquet files.

This operation may take a while.

In [21]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

24/02/27 09:23:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/27 09:23:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/27 09:23:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


Check the created files

In [22]:
!ls -lh fhvhv/2021/01/

total 226M
-rwxrwxrwx 1 root root    0 Feb 27 09:23 _SUCCESS
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00000-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00001-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00002-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00003-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00004-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00005-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00006-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00007-c3ecd058-94a4-4293-9035-2d563f26d342-c000.snappy.parquet
-rwxrwxrwx 1 root root 9.4M Feb 27 09:23 part-00008-c3ecd058-94a4-4

# Spark Dataframes

Create a dataframe from the parquet files.

In [23]:
df = spark.read.parquet('fhvhv/2021/01/')

Check the schema

In [24]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



# Functions and UDFs

In [25]:
from pyspark.sql import functions as F

In [26]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-21 18:20:09|2021-01-21 18:37:22|         234|         143|   null|
|           HV0005|              B02510|2021-01-19 20:32:04|2021-01-19 20:39:59|         130|         191|   null|
|           HV0005|              B02510|2021-01-09 01:51:34|2021-01-09 02:07:10|         119|         129|   null|
|           HV0003|              B02887|2021-01-22 05:09:12|2021-01-22 05:14:32|          76|          77|   null|
|           HV0003|              B02864|2021-01-01 03:25:05|2021-01-01 03:33:04|          69|         247|   null|
|           HV0005|              B02510|2021-01-21 16:40:16|2021-01-21 17:02:44|

Using built-in functions (`F.to_date()`)

In [27]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-21|  2021-01-21|         234|         143|
| 2021-01-19|  2021-01-19|         130|         191|
| 2021-01-09|  2021-01-09|         119|         129|
| 2021-01-22|  2021-01-22|          76|          77|
| 2021-01-01|  2021-01-01|          69|         247|
| 2021-01-21|  2021-01-21|         260|         234|
| 2021-01-28|  2021-01-28|          35|          47|
| 2021-01-24|  2021-01-24|         211|          79|
| 2021-01-05|  2021-01-05|         188|          85|
| 2021-01-16|  2021-01-16|         107|          49|
| 2021-01-30|  2021-01-30|         210|          97|
| 2021-01-14|  2021-01-14|         159|         146|
| 2021-01-11|  2021-01-11|          97|          48|
| 2021-01-24|  2021-01-24|         211|         237|
| 2021-01-26|  2021-01-26|         190|         181|
| 2021-01-30|  2021-01-30|         225|       

Creating a UDF

In [28]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [30]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-21|  2021-01-21|         234|         143|
|  e/9ce| 2021-01-19|  2021-01-19|         130|         191|
|  e/9ce| 2021-01-09|  2021-01-09|         119|         129|
|  e/b47| 2021-01-22|  2021-01-22|          76|          77|
|  e/b30| 2021-01-01|  2021-01-01|          69|         247|
|  e/9ce| 2021-01-21|  2021-01-21|         260|         234|
|  e/9ce| 2021-01-28|  2021-01-28|          35|          47|
|  e/9ce| 2021-01-24|  2021-01-24|         211|          79|
|  a/a7a| 2021-01-05|  2021-01-05|         188|          85|
|  a/b43| 2021-01-16|  2021-01-16|         107|          49|
|  e/9ce| 2021-01-30|  2021-01-30|         210|          97|
|  e/b3b| 2021-01-14|  2021-01-14|         159|         146|
|  e/b38| 2021-01-11|  2021-01-11|          97|          48|
|  e/b35| 2021-01-24|  2